<a href="https://colab.research.google.com/github/Zahra-Mhdi/Deep-Learning-Exercises/blob/main/Session_3_Exercise_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 3-A : Try other datasets : California Housing

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

data = fetch_california_housing()
X, y = data.data, data.target

X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_np = scaler.fit_transform(X_train_np)
X_test_np  = scaler.transform(X_test_np)

X_train = torch.tensor(X_train_np, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1).to(device)

class DeepNet(nn.Module):
  def __init__(self, input_dim, hidden_dims, output_dim, dropout_rate=0.4):
    super(DeepNet, self).__init__()
    self.layers = nn.Sequential(
        nn.Linear(input_dim, hidden_dims[0]),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(hidden_dims[0], hidden_dims[1]),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(hidden_dims[1], hidden_dims[2]),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(hidden_dims[2], output_dim)
    )
  def forward(self, x):
    return self.layers(x)

model = DeepNet(input_dim=X_train.shape[1], hidden_dims=[64, 32, 16], output_dim=1).to(device)
print(model)

loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
epochs = 50
for epoch in range(1, epochs + 1):
  model.train()
  preds = model(X_train)
  l = loss(preds, y_train)
  optimizer.zero_grad()
  l.backward()
  optimizer.step()
  print(f"[California] Epoch {epoch:03d}/{epochs} - Loss: {l.item():.4f}")


DeepNet(
  (layers): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.4, inplace=False)
    (6): Linear(in_features=32, out_features=16, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.4, inplace=False)
    (9): Linear(in_features=16, out_features=1, bias=True)
  )
)
[California] Epoch 001/50 - Loss: 6.4087
[California] Epoch 002/50 - Loss: 6.1275
[California] Epoch 003/50 - Loss: 5.8577
[California] Epoch 004/50 - Loss: 5.6073
[California] Epoch 005/50 - Loss: 5.3668
[California] Epoch 006/50 - Loss: 5.1348
[California] Epoch 007/50 - Loss: 4.9158
[California] Epoch 008/50 - Loss: 4.6929
[California] Epoch 009/50 - Loss: 4.4892
[California] Epoch 010/50 - Loss: 4.2903
[California] Epoch 011/50 - Loss: 4.0897
[California] Epoch 012/50 - Loss: 3.9134
[California] Epoch 013/50 - Loss: 3.7228
[California] Epoch 014/50

Exercise 3-B : Try other datasets : MNIST

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

transform = transforms.ToTensor()
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

class DeepNet(nn.Module):
  def __init__(self):
    super(DeepNet, self).__init__()
    self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, 256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(64, 10)
    )
  def forward(self, x):
    return self.layers(x)

model = DeepNet().to(device)
print(model)

loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
epochs = 1
for epoch in range(1, epochs + 1):
  model.train()
  total_loss = 0.0
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    logits = model(images)
    l = loss(logits, labels)
    l.backward()
    optimizer.step()
    total_loss += l.item()
  print(f"[MNIST] Epoch {epoch:03d}/{epochs} - Avg Loss: {total_loss/len(train_loader):.4f}")


DeepNet(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=128, out_features=64, bias=True)
    (8): ReLU()
    (9): Dropout(p=0.4, inplace=False)
    (10): Linear(in_features=64, out_features=10, bias=True)
  )
)
[MNIST] Epoch 001/1 - Avg Loss: 2.2827
